# Database Migration Runner

**Filename:** `migrations/migration_runner_notebook.py`

**Description:** This notebook provides a safe and reliable way to apply schema changes to the `workouts.db` database. \
It automates the process of backing up the database, applying new SQL migration scripts, and tracking which scripts have \
 already been run to prevent accidental re-runs.

### How to Use
This notebook is the final step in the migration process. It is designed to run `.sql` files that you create in the \
`migrations` folder.

### Your Workflow
**1. Create a Migration File:**
- First, create your migration logic in a new `.sql` file inside the `migrations/` directory.
- Use the `000_migration_template.sql` as a starting point.
- Name your file sequentially (e.g., `006_add_new_feature.sql`).

**2. Run This Notebook**
- Once your `.sql` file is ready, open this notebook (`migration_runner_notebook.ipynb`).
- Run the cells sequentially from top to bottom.
- The script will automatically find your new `.sql` file, back up the database, and execute the migration.

### Key Features
- **Automated Backups:** Creates a timestamped backup in the `../backups/` folder before any changes are made.
- **Idempotent:** The script keeps track of applied migrations in a `_migrations table`, so it will never run the \
same script twice.
- **Transactional:** Each migration is wrapped in a transaction. If any part of a script fails, the entire change is \
automatically rolled back, leaving your database in a clean state.

## Run Code
---

### Cell 1: Imports and Configuration

This cell imports all necessary libraries and sets up the key configuration variables for the script.

In [86]:
# cell 1: Imports and Configuration
import sqlite3
import os
import shutil
from datetime import datetime

# --- Configuration ---
# The path to your SQLite database file relative to the 'migrations' directory.
DB_FILE = "/Volumes/Public/Container_Settings/sqlpage/www/workouts.db"

# The directory where your .sql migration files are located.
# MIGRATIONS_DIR = "./"  # Assumes the script/notebook is in the 'migrations' directory
MIGRATIONS_DIR = "/Volumes/Public/Container_Settings/sqlpage/migrations"

# The directory where backups will be stored.
BACKUP_DIR = "/Volumes/Public/Container_Settings/sqlpage/backups"

### Cell 2: Helper Functions

This cell defines all the utility functions that the main migration process will use. This includes functions\
for backing up the database, connecting to it, and tracking which migrations have already been applied. 

In [87]:
# cell 2: Helper Functions
def backup_database():
    """Creates a timestamped backup of the database file."""
    if not os.path.exists(BACKUP_DIR):
        os.makedirs(BACKUP_DIR)
    
    timestamp = datetime.now().strftime("%Y-%m-%d_%H%M%S")
    backup_file_name = f"workouts-backup-{timestamp}.db"
    backup_path = os.path.join(BACKUP_DIR, backup_file_name)
    
    try:
        print(f"Backing up database to {backup_path}...")
        shutil.copyfile(DB_FILE, backup_path)
        print("Backup successful.")
        return True
    except Exception as e:
        print(f"ERROR: Database backup failed: {e}")
        return False

def get_db_connection():
    """Establishes a connection to the SQLite database."""
    return sqlite3.connect(DB_FILE)

def get_applied_migrations(conn):
    """
    Gets the list of migrations that have already been applied by checking
    a special '_migrations' table. Creates the table if it doesn't exist.
    """
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS _migrations (
            filename TEXT PRIMARY KEY,
            applied_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)
    cursor.execute("SELECT filename FROM _migrations")
    return {row[0] for row in cursor.fetchall()}

def record_migration(conn, filename):
    """Records a migration as applied in the database."""
    cursor = conn.cursor()
    cursor.execute("INSERT INTO _migrations (filename) VALUES (?)", (filename,))
    conn.commit()

### Cell 3: Main Migration Logic
This cell contains the main `run_migrations` function. It orchestrates the entire process:\
backing up the database, finding new migration files, executing them, and recording the results.

In [88]:
# cell 3: Migration Execution Logic
def run_migrations():
    """
    Finds and applies all pending .sql migration scripts in the directory.
    """
    print("Starting migration process...")

    # Step 1: Backup the database first as a safety measure
    if not backup_database():
        return # Stop if backup fails

    conn = get_db_connection()
    applied_migrations = get_applied_migrations(conn)
    
    # Step 2: Find all .sql files in the directory
    migration_files = sorted([
        f for f in os.listdir(MIGRATIONS_DIR)
        if f.endswith('.sql') and f != '000_migration_template.sql'        
    ])

    found_new_migration = False
    for filename in migration_files:
        if filename not in applied_migrations:
            found_new_migration = True
            print(f"\nApplying migration: {filename}...")
            try:
                with open(os.path.join(MIGRATIONS_DIR, filename), 'r') as f:
                    sql_script = f.read()
                
                # Step 3: Execute the entire SQL script within a transaction
                conn.executescript(sql_script)
                
                # Step 4: Record the migration as successful
                record_migration(conn, filename)
                print(f"Successfully applied {filename}.")
                
            except Exception as e:
                print(f"!!!!!!!! ERROR applying {filename} !!!!!!!!")
                print(f"Error details: {e}")
                print("Migration process halted. The database has been rolled back.")
                conn.rollback() # Rollback any partial changes from the failed script
                conn.close()
                return

    if not found_new_migration:
        print("\nDatabase is already up to date. No new migrations found.")

    conn.close()
    print("\nMigration process finished.")

### Cell 4: Execute the Migration

This is the final cell. Running this cell will trigger the entire migration process by calling the\
`run_migrations` function.

In [89]:
# cell 4: Run the Migration Process
run_migrations()

Starting migration process...
Backing up database to /Volumes/Public/Container_Settings/sqlpage/backups/workouts-backup-2025-07-03_133729.db...
Backup successful.

Applying migration: 013_create_default_admin_user.sql...
!!!!!!!! ERROR applying 013_create_default_admin_user.sql !!!!!!!!
Error details: near "(": syntax error
Migration process halted. The database has been rolled back.
